#### 문제 2-2 : 영화 리뷰 감정 분석기

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_w


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립


In [8]:
# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

print("프롬프트 템플릿 생성 완료")

프롬프트 템플릿 생성 완료


In [11]:
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",  
    temperature=0.7
)

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

print("모델 및 파서 설정 완료")

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",  
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!", 
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",  
    "영상미는 좋았지만 연기가 아쉬웠어요.",              
    "최고의 작품이었습니다! 다시 보고 싶어요."            
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 5개 준비 완료


In [10]:
# 안전한 감정 분석 함수
def safe_sentiment_analysis(text, use_fixing_parser=True):
    try:
        chain = prompt | llm | (fixing_parser if use_fixing_parser else parser)
        result = chain.invoke({"text": text})
        return result, None
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행
print("=== 실제 감정 분석 결과 ===")

success_count = 0
total_count = len(texts)

for i, text in enumerate(texts, 1):
    print(f"\n{i}. 텍스트: {text}")

    result, error = safe_sentiment_analysis(text, use_fixing_parser=True)

    if result:
        print(f"   감정: {result.value}")
        success_count += 1
    else:
        print(f"   오류: {error}")
        print("   기본 파서로 재시도...")

        result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
        if result2:
            print(f"   감정: {result2.value} (기본 파서 성공)")
            success_count += 1
        else:
            print(f"   재시도 실패: {error2}")

print(f"\n=== 결과 요약 ===")
print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
print(f"실패: {total_count-success_count}/{total_count}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 중립

4. 텍스트: 영상미는 좋았지만 연기가 아쉬웠어요.
   감정: 부정

5. 텍스트: 최고의 작품이었습니다! 다시 보고 싶어요.
   감정: 긍정

=== 결과 요약 ===
성공: 5/5 (100.0%)
실패: 0/5
